En este informe se explican las queries realizadas sobre los índices `wine`, `inn`, `maraton`, y `norte`, cuyo indexado en el servidor de Elasticsearch se realiza con el script `document-indexing.py`.

Se muestra tanto el código necesario para realizarlas con la API REST de Elasticsearch (en Python) como el resultado obtenido.

In [38]:
import json
import requests
from datetime import datetime

ELASTIC = "http://localhost:9200" #dirección en la que está funcionando elastic

# Índice Wine

Este índice contiene datos sobre distintos vinos. 

## Mejores 5 vinos de cada color

Esta query busca los 5 vinos con mejor puntuación de cada color (*White* y *Red*). Se ha realizado mediante dos queries, una que incluye los vinos tintos y otra que incluye los blancos. Los resultados se limitan a los primeros 5 elementos ordenados según el campo **Score**.

In [16]:
query = {
    "size" : 5,
    "query":{
        "match" : { "Color" : "Red" }
    },
    "sort" : {
        "Score" : "desc"
    }
}

reds = json.loads(requests.get(ELASTIC+"/wine/_search",json=query).text)["hits"]["hits"]

query = {
    "size" : 5,
    "query": {
        "match" : { "Color" : "White" }
    },
    "sort" : {
        "Score" : "desc"
    }
}

whites = json.loads(requests.get(ELASTIC+"/wine/_search",json=query).text)["hits"]["hits"]

In [18]:
print("Vinos tintos:")
print("----------------")
for i, w in enumerate(reds):
    wine = w["_source"]
    print("#%d: %s, %d/100" % (i+1, wine["Name"], wine["Score"]))
print("")

print("Vinos blancos:")
print("----------------")
for i, w in enumerate(whites):
    wine = w["_source"]
    print("#%d: %s, %d/100" % (i+1, wine["Name"], wine["Score"]))
print("")

Vinos tintos:
----------------
#1: A Shot in the Dark Eleven Confessions Vineyard, 98/100
#2: Papas Block, 98/100
#3: In the Crosshairs Eleven Confessions Vineyard, 97/100
#4: Cabernet Sauvignon, 97/100
#5: Beckstoffer Dr. Crane Vineyard, 97/100

Vinos blancos:
----------------
#1: Green River Isobel, 95/100
#2: Boonflys Hill, 95/100
#3: McCrea Vineyard, 95/100
#4: Ma Belle-Fille, 95/100
#5: Durell Vineyard, 95/100



## Precio medio y puntuación máxima del vino por tipo de uva

Esta query divide los vinos por su tipo de uva, y para cada uva calcula la media de su precio y el máximo de puntuación obtenido por los vinos de dicha uva. Se ha realizado mediante un *aggregation* que primero divide los elementos por su campo **Grape**, y dentro de cada división realiza dos *aggregations*, una calcula *avg* del campo **Price** y otra *max* del campo **Score**.

In [20]:
query = { 
    "aggs" : {
        "uvas" : {
            "terms" : {
                "field" : "Grape.keyword"
            },
            "aggs":{
                "precio-medio-por-uva": {
                    "avg" : {
                        "field" : "Price"
                    }
                },
                "puntuacion-maxima-por-uva" : {
                    "max" : {
                        "field" : "Score"
                    }
                }
            }
        } 
    }
}

uvas = json.loads(requests.get(ELASTIC+"/wine/_search",json=query).text)["aggregations"]["uvas"]["buckets"]

In [21]:
for obj in uvas:
    print("%s: %d vinos" % (obj["key"], obj["doc_count"]))
    print("Precio medio: $%0.2f" % (obj["precio-medio-por-uva"]["value"]))
    print("Puntuación máxima: %d/100" % (obj["puntuacion-maxima-por-uva"]["value"]))
    print("")

Pinot Noir: 109 vinos
Precio medio: $46.98
Puntuación máxima: 95/100

Chardonnay: 104 vinos
Precio medio: $41.93
Puntuación máxima: 95/100

Zinfandel: 78 vinos
Precio medio: $28.94
Puntuación máxima: 95/100

Cabernet Sauvingnon: 68 vinos
Precio medio: $81.29
Puntuación máxima: 97/100

Syrah: 61 vinos
Precio medio: $44.95
Puntuación máxima: 98/100

Sauvignon Blanc: 46 vinos
Precio medio: $24.02
Puntuación máxima: 93/100

Grenache: 10 vinos
Precio medio: $68.80
Puntuación máxima: 97/100

Merlot: 10 vinos
Precio medio: $30.20
Puntuación máxima: 89/100

Petite Sirah: 6 vinos
Precio medio: $41.83
Puntuación máxima: 91/100

Barbera: 1 vinos
Precio medio: $17.00
Puntuación máxima: 87/100



## Precio medio de vinos producidos antes de 2007

Esta query encuentra los vinos producidos en años anteriores a 2007, y calcula el precio medio. Se ha realizado mediante una *query* que restrinje el campo **Year** a valores menores que 2007, y una *aggregation* que calcula la media del campo **Price**.

In [29]:
query = {
    "query":{
        "range" : { 
            "Year" : {
                "lt" : 2007
            }
        }
    },
    "aggs":{
        "precio-medio": {
            "avg" : {
                "field" : "Price"
            }
        }
    }
}

precio = json.loads(requests.get(ELASTIC+"/wine/_search",json=query).text)["aggregations"]["precio-medio"]["value"]

In [31]:
print("Precio medio de los vinos anteriores a 2007: $%0.2f" % precio)

Precio medio de los vinos anteriores a 2007: $53.37


# Índice Inn

Este índice contiene información sobre distintas reservas realizadas en un motel.

## Reservas de duración mayor a 10 días

Esta query busca aquellas reservas que hayan durado más de una semana y media (redondeado a 10 días). Para ello se ha utilizado una query con script, un tipo especial de query que permite la utilización de código que devuelva verdadero o falso, condición que será aplicada a cada documento, devolviendo aquellos donde la condición es cierta. En este caso la condición es que la diferencia entre la fecha **CheckOut** y **CheckIn** sea mayor que 10, expresada en días. 

Elasticsearch no permite operaciones directamente con fechas, por lo que pasamos los valores a milisegundos, y tras la resta lo volvemos a convertir en días.

In [48]:
query = {
    "size": 10000,
    "query":
    {
        "script" : {
            "script" : {
                "source": "return (doc['CheckOut'].value.getMillis() - doc['CheckIn'].value.getMillis())/(1000*60*60*24) > 10"
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["hits"]["hits"]

In [49]:
for r in res:
    reservation = r["_source"]
    ckin = datetime.strptime(reservation["CheckIn"], "%d-%b-%y")
    ckout = datetime.strptime(reservation["CheckOut"], "%d-%b-%y")

    print("%s %s: Entró %s, Salió %s, duración = %d días" % (reservation["FirstName"], reservation["LastName"],reservation["CheckIn"],
                                                             reservation["CheckOut"],(ckout-ckin).days))

print("")

LEWIS TRUDEN: Entró 21-Jun-10, Salió 02-Jul-10, duración = 11 días
GUS DERKAS: Entró 01-Oct-10, Salió 13-Oct-10, duración = 12 días
OLYMPIA ALBROUGH: Entró 14-Oct-10, Salió 25-Oct-10, duración = 11 días
LESTER PANOS: Entró 22-Nov-10, Salió 04-Dec-10, duración = 12 días
DENNY PERRINO: Entró 07-Mar-10, Salió 19-Mar-10, duración = 12 días
ELMIRA ATTEBURG: Entró 21-Nov-10, Salió 02-Dec-10, duración = 11 días
ISREAL BISHOFF: Entró 01-Jun-10, Salió 12-Jun-10, duración = 11 días
MAXIE STEBNER: Entró 15-Jul-10, Salió 28-Jul-10, duración = 13 días
PORTER STRACK: Entró 02-Sep-10, Salió 14-Sep-10, duración = 12 días
HERB CRACE: Entró 06-Jan-10, Salió 18-Jan-10, duración = 12 días
MARINE CASMORE: Entró 22-May-10, Salió 03-Jun-10, duración = 12 días
RODERICK BRODOWSKI: Entró 12-Jan-10, Salió 25-Jan-10, duración = 13 días
ASUNCION TIPPIN: Entró 24-Apr-10, Salió 06-May-10, duración = 12 días
FRANCISCO BOSE: Entró 03-Oct-10, Salió 14-Oct-10, duración = 11 días
GRANT KNERIEN: Entró 12-Dec-10, Salió 25-

## Personas que se hayan alojado más de una vez

Esta query busca personas con más de una reserva en el sistema. Para ello se ha utilizado una *aggregation* que clasifica los documentos en grupos por el nombre completo (realizable mediante un script que une los campos **FirstName** y **LastName** en uno), y para cada clasificación, implementa una *aggregation* de tipo *bucket_selector*, que permite obtener la cuenta de documentos en cada grupo, y quedarse sólo con aquellos grupos donde dicha cuenta sea mayor que 1.

In [52]:
query = {
    "size":0,
    "aggs": {
        "persona":{
            "terms" : {
                "script": "params['_source'].FirstName+' '+params['_source'].LastName"
            },
            "aggs":{
                "reservas":{
                    "bucket_selector": {
                        "buckets_path": {
                            "cuenta" : "_count"
                        },
                        "script": {
                            "source" : "params.cuenta > 1"
                        }
                    }
                }
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["aggregations"]["persona"]["buckets"]

In [53]:
for r in res:
    print("%s se ha alojado %d veces" % (r["key"], r["doc_count"]))

EMERY VOLANTE se ha alojado 3 veces
GLEN DONIGAN se ha alojado 3 veces
GRANT KNERIEN se ha alojado 3 veces
ALEXIS FINEFROCK se ha alojado 2 veces
AMIEE PENEZ se ha alojado 2 veces
ARON KEBEDE se ha alojado 2 veces
AUGUST MAEWEATHER se ha alojado 2 veces
BENTON AKHTAR se ha alojado 2 veces
BO DURAN se ha alojado 2 veces
CATHERIN KUDRON se ha alojado 2 veces


## Número de reservas al mes para camas de tipo *Queen*

Esta query devuelve el número de reservas al mes en habitaciones que tengan cama tipo *Queen*. Para ello se realiza una búsqueda de las reservas cuya habitación tenga el tipo de cama *Queen* mediante el campo **bedType** dentro del objeto **Room**, y sobre estos resultados se realiza una *aggregation* en la que se agrupan los resultados en grupos según el mes de la reserva, que se puede obtener mediante un script, que llama al método *getMonth* aplicado al campo de tipo fecha **CheckIn**.

In [54]:
query = {
    "size" : 0,
    "query" : {
        "match" : { "Room.bedType" : "Queen" }
    },
    "aggs" : {
        "reservas-al-mes" : {
            "terms" : { 
                "script" : "doc['CheckIn'].value.getMonth()"
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["aggregations"]["reservas-al-mes"]["buckets"]

In [55]:
for r in res:
    print("En el mes %s: %d reservas de camas de tipo Queen" % (r["key"],r["doc_count"]))

En el mes AUGUST: 25 reservas de camas de tipo Queen
En el mes DECEMBER: 24 reservas de camas de tipo Queen
En el mes APRIL: 21 reservas de camas de tipo Queen
En el mes JULY: 21 reservas de camas de tipo Queen
En el mes OCTOBER: 21 reservas de camas de tipo Queen
En el mes NOVEMBER: 19 reservas de camas de tipo Queen
En el mes SEPTEMBER: 19 reservas de camas de tipo Queen
En el mes JUNE: 18 reservas de camas de tipo Queen
En el mes MAY: 18 reservas de camas de tipo Queen
En el mes JANUARY: 16 reservas de camas de tipo Queen


## Precio total de las reservas realizadas por *EMERY VOLANTE*

Esta query calcula el precio total de las reservas del cliente EMERY VOLANTE, que como vimos anteriormente se ha alojado 3 veces. Esta query consiste en buscar las reservas cuyo campo **FirstName** sea EMERY y **LastName** sea VOLANTE, y sobre el resultado aplicar una *aggregation* que sume el coste de las reservas, calculado con un script que multiplica el campo **Rate** (precio de habitación por día) por los días que se ha alojado (calculado de la misma manera que en la primera query).

In [57]:
query = {
    "query" : {
        "bool" : {
            "must" : [{"match" : { "FirstName" : "EMERY"}}, {"match" : { "LastName" : "VOLANTE"}}]
        }
    },
    "aggs" : {
       "precio-total" : {
           "sum" : {
               "script" : {
                   "source" : "(doc.CheckOut.value.getMillis() - doc.CheckIn.value.getMillis())/(1000*60*60*24) * doc.Rate.value" 
               }
           }
       } 
    }
}

res = json.loads(requests.get(ELASTIC+"/inn/_search",json=query).text)["aggregations"]

In [59]:
print("Precio total de todas las reservas realizadas por EMERY VOLANTE: $%0.2f" % res["precio-total"]["value"])
print("")

Precio total de todas las reservas realizadas por EMERY VOLANTE: $843.75



# Índice Maraton

Este índice contiene varios datos sobre los corredores de una maratón.

## Media, mejor y peor tiempo por grupo de edad

Esta query devuelve, por cada grupo de edad, la media, mínimo, y máximo tiempo obtenido. Para ello se realiza una *aggregation* que divide los documentos en los distintos grupos de edad, y para cada grupo se aplica, sobre el campo **Time** operaciones *avg*, *min* y *max*.

In [63]:
query = {
    "size" : 0,
    "aggs" : {
        "tiempos_por_grupo" : {
            "terms" : {
                "field" : "Group.keyword"
            },
            "aggs" : {
                "tiempos" : {
                    "avg" : {
                        "field" : "Time"
                    }
                },
                "mejor-tiempo" : {
                    "min" : {
                        "field" : "Time"
                    }
                },
                "peor-tiempo" : {
                    "max" : {
                        "field" : "Time"
                    }
                }
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["aggregations"]["tiempos_por_grupo"]["buckets"]

In [64]:
for r in res:
    
    print("Grupo de edad %s: %d corredores, tiempo medio %s, \
tiempo perdedor: %s, tiempo ganador: %s" % (r["key"], r["doc_count"],
                                                r["tiempos"]["value_as_string"],
                                                r["peor-tiempo"]["value_as_string"],
                                                r["mejor-tiempo"]["value_as_string"]))

Grupo de edad 20-39: 272 corredores, tiempo medio 1:47:28, tiempo perdedor: 2:41:04, tiempo ganador: 1:09:47
Grupo de edad 40-49: 139 corredores, tiempo medio 1:46:51, tiempo perdedor: 2:37:23, tiempo ganador: 1:18:36
Grupo de edad 50-59: 38 corredores, tiempo medio 1:51:09, tiempo perdedor: 2:27:23, tiempo ganador: 1:26:51
Grupo de edad 60-98: 7 corredores, tiempo medio 1:54:26, tiempo perdedor: 2:05:21, tiempo ganador: 1:37:56
Grupo de edad 01-19: 5 corredores, tiempo medio 1:44:14, tiempo perdedor: 2:04:02, tiempo ganador: 1:21:24
Grupo de edad 99-+: 2 corredores, tiempo medio 2:01:10, tiempo perdedor: 2:22:11, tiempo ganador: 1:40:09


## Mejor posición por Estado

Esta query devuelve, por cada estado, la mínima posición obtenida. Para ello se realiza una *aggregation* que divide los documentos en los distintos estados, y para cada grupo se calcula el *min* del campo **Place**.

In [65]:
query = {
    "size" : 0,
    "aggs" : {
        "estados" : {
            "terms" : {
                "field" : "State.keyword"
            },
            "aggs" : {
                "mejor" : {
                    "min" : {
                        "field" : "Place"
                    }
                },
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["aggregations"]["estados"]["buckets"]

In [66]:
for r in res:
    print("%s: %d corredores, mejor posición %d" % (r["key"], r["doc_count"], r["mejor"]["value"]))

RI: 190 corredores, mejor posición 3
MA: 182 corredores, mejor posición 1
CT: 32 corredores, mejor posición 20
NH: 20 corredores, mejor posición 12
FL: 9 corredores, mejor posición 17
NC: 6 corredores, mejor posición 54
MO: 5 corredores, mejor posición 7
IN: 4 corredores, mejor posición 58
NJ: 4 corredores, mejor posición 69
PA: 4 corredores, mejor posición 100


## Corredores con ritmo entre 6 y 8 minutos/km que hayan quedado por encima del puesto 5 en su grupo

Esta query encuentra los corredores con ritmos en el rango entre 6 y 8 minutos que además hayan quedado por encima del puesto 5 para su grupo de edad. Para ello se utilizan dos condiciones en la búsqueda, que el campo **Pace** sea mayor o igual que 6 minutos y menor o igual que 8 minutos, y que **GroupPlace** sea menor o igual que 5.

In [67]:
query = {
    "size": 10000,
    "query" : {
        "bool" : {
            "must" : [{
                "range" : { 
                    "Pace" : {
                        "gte" : "0:6:00",
                        "lt" : "0:8:00"
                    }
                }},
                {"range" : {
                    "GroupPlace" : {
                        "lte" : 5
                    }
                }
            }]
        }
    }
}


res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["hits"]["hits"]

In [68]:
for re in res:
    r = re["_source"]
    print("%s, Pace: %s, Puesto en su grupo(%s) %d" % (r["FirstName"]+" "+r["LasName"], r["Pace"], r["Group"], r["GroupPlace"]))

CRUZ TAILOR, Pace: 0:6:00, Puesto en su grupo(40-49) 1
BRENTON FAGO, Pace: 0:6:03, Puesto en su grupo(40-49) 2
CHARLES FARLESS, Pace: 0:6:03, Puesto en su grupo(40-49) 3
WILBER VANORDER, Pace: 0:6:04, Puesto en su grupo(40-49) 4
KENDRICK HOLZ, Pace: 0:6:13, Puesto en su grupo(01-19) 1
JAMA PEAD, Pace: 0:6:16, Puesto en su grupo(20-39) 1
ROMEO UNVARSKY, Pace: 0:6:18, Puesto en su grupo(40-49) 5
ALEXIS HABERMANN, Pace: 0:6:38, Puesto en su grupo(50-59) 1
MAGGIE NASR, Pace: 0:6:39, Puesto en su grupo(20-39) 2
FRANCIS LAMSON, Pace: 0:6:42, Puesto en su grupo(50-59) 2
HEATH KIRVIN, Pace: 0:7:02, Puesto en su grupo(50-59) 3
NEDRA NASSIMI, Pace: 0:7:03, Puesto en su grupo(20-39) 3
FLORIDA AYTES, Pace: 0:7:03, Puesto en su grupo(40-49) 1
LETTIE MCKAGUE, Pace: 0:7:05, Puesto en su grupo(20-39) 4
MAIRA ROUTHIER, Pace: 0:7:11, Puesto en su grupo(20-39) 5
GIOVANNI FYLES, Pace: 0:7:17, Puesto en su grupo(50-59) 4
STEPHANI FERTITTA, Pace: 0:7:18, Puesto en su grupo(40-49) 2
WESLEY BECKWORTH, Pace: 0

## Número de BIB de los diez mejores corredores

Esta query devuelve el número de BIB de los 10 mejores corredores. Para ello se realiza una búsqueda de los documentos donde el campo **Place** sea menor o igual que 10.

In [69]:
query = {
    "size": 10,
    "query" : {
        "range" : {
            "Place" : {
                "lte" : "10"
            }
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/maraton/_search",json=query).text)["hits"]["hits"]

In [71]:
for re in res:
    r = re["_source"]
    print("%s, puesto %s, BIB: %s" % (r["FirstName"]+" "+r["LasName"], r["Place"], r["BIBNumber"]))

ANDRE PELLAM, puesto 1, BIB: 340
HORACE KARPOWICZ, puesto 2, BIB: 34
MILES ROSELLA, puesto 3, BIB: 399
FRANKLYN GIRARDIN, puesto 4, BIB: 18
AL LEMASTER, puesto 5, BIB: 262
CRUZ TAILOR, puesto 6, BIB: 308
DEWEY HEAIVILIN, puesto 7, BIB: 45
HYMAN CURIE, puesto 8, BIB: 16
BRENTON FAGO, puesto 9, BIB: 42
CHARLES FARLESS, puesto 10, BIB: 420


# Índice norte

Este índice contiene noticias publicadas por el Norte de Castilla durante el año 2006.

## Artículos de la sección "Internacional" que contengan la frase "crisis económica"

En esta query se buscan los artículos cuya sección sea Internacional que en el cuerpo de la noticia contengan la frase "crisis económica". Para ello se realiza una búsqueda con dos condiciones, que el campo **seccion** sea Internacional, y que el campo **cuerpo** contenga la frase "crisis económica".

In [73]:
query = {
    "size" : 10000,
    "query" : {
        "bool" : {
            "must" : [
                {"match" : {"seccion" : "Internacional"}},
                {"match_phrase" : {"cuerpo" : "crisis económica"}}
            ]
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/norte/_search",json=query).text)["hits"]["hits"]

In [74]:
for re in res:
    r = re["_source"]
    
    print('"%s" [score %0.2f]' % (r["titulo"],re["_score"]))
    
    if "resumen" in r:
        print("\t"+r["resumen"])
    print("")

"Los argentinos reciben con indiferencia la orden por la que los bancos devolverán en pesos los ahorros retenidos" [score 10.76]
	Los afectados, insatisfechos con la solución, insistirán en sus demandas

"Las empresas españolas en América Latina temen nuevos cambios legales" [score 9.60]
	La inseguridad jurídica es la principal razón que alegan las compañías para paralizar o reconsiderar sus inversiones en estos países

"Berlusconi acusa a los empresarios de aliarse con la oposición" [score 9.14]
	La patronal italiana califica las palabras del primer ministro de «antidemocráticas»

"Ferviente nacionalista crítico con WashingtonEl peor dirigente de la historia del país" [score 8.15]

"Perú da otra oportunidad a Alan García, que dejó el país en bancarrota en 1990" [score 8.15]
	El futuro presidente pide perdón por sus errores del pasado, de los que culpa al «apetito de poder», y arremete contra la injerencia de Chávez

"Una segunda oleada de vandalismo y enfrentamientos sacude Budapest" 

## Artículos publicados tal día como hoy en 2006 dentro de la sección Televisión

En esta query se buscan los artículos que se hayan publicado en dd-MM-2006, siendo dd y MM el día y mes correspondiente al actual, y estén en la sección de Televisión. Para ello se obtiene el día y mes actual, y se realiza una búsqueda con dos condiciones, que el campo **seccion** sea Televisión, y que el campo **fecha** sea dd-MM-2006.

In [75]:
day = datetime.now().date().day
month = datetime.now().date().month

query = {
    "size" : 10000,
    "query" : {
        "bool" : {
            "must" : [
                {"match" : { "seccion" : "Televisión" }},
                {"match" : { "fecha" : str(day).zfill(2)+"-"+str(month).zfill(2)+"-2006" }}
            ]
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/norte/_search",json=query).text)["hits"]["hits"]

In [76]:
for re in res:
    r = re["_source"]
    
    print('"%s"' % (r["titulo"]))
    
    if "resumen" in r:
        print("\t"+r["resumen"])
    print("")

"«Aída es la mujer común de nuestra sociedad»"
	La cadena de Mediaset estrena esta noche una nueva temporada de la comedia, que vuelve con novedades

"Superhuman"

"Finaliza el espacio 'Esta es mi tierra'"

"Debuta en Cuatro la versión española de 'Matrimonio con hijos'"

"'Actualidad en 2D' trata el islamismo"



## Artículos publicados durante el mes de mayo que contengan la palabra "Eurovisión"

En esta query se buscan los artículos que contengan la palabra "eurovisión" entre los artículos publicados en Mayo. Para ello se realiza una búsqueda con dos condiciones, que el campo **cuerpo** contenga la palabra eurovisión y que la fecha sea mayor o igual que 05-2006, lo que se traduce a mayor o igual que el 1 de Mayo de 2006, y menor que 06-2006, es decir, el 1 de Junio de 2006.

In [79]:
query = {
    "size" : 10000,
    "query" : {
        "bool" : {
            "must" : [
                {"match" : { "cuerpo" : "eurovisión" }},
                {"range" : { 
                    "fecha" : {
                        "gte" : "05-2006",
                        "lt" : "06-2006",
                        "format" : "MM-yyyy"
                    }
                }}
            ]
        }
    }
}

res = json.loads(requests.get(ELASTIC+"/norte/_search",json=query).text)["hits"]["hits"]

In [80]:
for re in res:
    r = re["_source"]
    
    print('"%s"' % (r["titulo"]))
    if "resumen" in r:
        print("\t"+r["resumen"])
    print("")

"«Lo único que queremos es hacerlo muy bien»"
	Las cuatro hermanas se muestran sorprendidas por el montaje del festival en la capital griega

"Todas la canciones de 'Eurovisión'"

"Las Ketchup visitan a Buenafuente"

"Bochorno"

"Casi cinco millones de espectadores vieron Eurovisión"

"Eurovisión"

"Carlos Lozano presentará en TVE1 el día 20 la gala del festival de Eurovisión"
	El cuarteto cordobés Las Ketchup parte mañana hacia Atenas para defender 'Bloody Mary'

"«Intento disfrutar de cada momento, no miro más allá de mi sombra»"
	La cantante recibe esta noche en el Museo Patio Herreriano de Valladolid un homenaje de la Academia de los Grammy Latinos

"Azehos impulsa la gastronomía con el primer festival de la tapa"
	Un total de 36 establecimientos se inscriben en el concurso para potenciar los productos de la tierra Con el aperitivo como excusa, la organización invita a zamoranos y visitantes a seguir la ruta gastronómica

